In [117]:
import numpy as np
import pandas as pd

In [118]:
df = pd.read_csv("./Data/dataset_mood_smartphone.csv")
df.head()
df.nans()

AttributeError: 'DataFrame' object has no attribute 'nans'

In [119]:
variables = np.unique(df.variable.values)
p = variables.shape[0]

time_points = np.unique(df.time.values)
T = time_points.shape[0]

n_response = np.sum([df.variable.values == 'mood'])

ids = np.unique(df.id.values)
n_ids = ids.shape[0]

id_df_list = [df[df.id == ids[i]] for i in range(n_ids)]

In [120]:
print(p, T, n_response, n_ids)

19 336907 5641 27


In [122]:
def df_timesplit(df):
    ret_df = df
    ret_df.insert(2, 'date', [ret_df.iloc[i].time[0:10] for i in range(len(ret_df))])
    ret_df.insert(3, 'hour', [ret_df.iloc[i].time[11:] for i in range(len(ret_df))])
    ret_df.drop(['time'], axis=1)
    return ret_df

In [123]:
alt_df = df_timesplit(id_df_list[1])




In [124]:
alt_df.head()

,Unnamed: 0,id,date,hour,time,variable,value
222,223,AS14.02,2014-03-16,15:00:00.000,2014-03-16 15:00:00.000,mood,7.0
223,224,AS14.02,2014-03-16,18:00:00.000,2014-03-16 18:00:00.000,mood,6.0
224,225,AS14.02,2014-03-16,21:00:00.000,2014-03-16 21:00:00.000,mood,6.0
225,226,AS14.02,2014-03-17,09:00:00.000,2014-03-17 09:00:00.000,mood,7.0
226,227,AS14.02,2014-03-17,12:00:00.000,2014-03-17 12:00:00.000,mood,8.0
